In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import googlemaps
import openai
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from transformers import pipeline
import scipy.io.wavfile as wavfile
from pyngrok import ngrok
import os
import base64
from io import BytesIO
import logging

# Apply nest_asyncio for asynchronous execution in Colab
nest_asyncio.apply()



# Access environment variables
google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
ngrok_auth_token = os.getenv("NGROK_AUTH_TOKEN")

# Validate that the keys are loaded properly
if not google_api_key or not openai_api_key or not ngrok_auth_token:
    raise ValueError("Missing environment variables: GOOGLE_API_KEY, OPENAI_API_KEY, or NGROK_AUTH_TOKEN.")

# Set your ngrok authtoken
ngrok.set_auth_token(ngrok_auth_token)

# Initialize FastAPI app
app = FastAPI()

# Pydantic model for input validation
class Location(BaseModel):
    latitude: float
    longitude: float

# Initialize logging
logging.basicConfig(filename='music_generation.log', level=logging.INFO)

# 1. Initialize GCP Geocoding client
def get_location_description(latitude, longitude):
    try:
        gmaps = googlemaps.Client(key=google_api_key)  # Use environment variable for your Google API key
        reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))
        if reverse_geocode_result:
            return reverse_geocode_result[0]["formatted_address"]
        return "Unknown location"
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error in geocoding: {str(e)}")

# 2. Generate concise prompt using GPT-4
def generate_soundscape_prompt(location_description):
    try:
        openai.api_key = openai_api_key  # Use environment variable for OpenAI API key

        prompt_template = PromptTemplate(
            input_variables=["location_description"],
            template=(
                "Based on the location: {location_description}, create a concise and creative prompt for MusicGen. "
                "The prompt should describe instrumental music with a soothing melody for therapy, reflecting the location."
            ),
        )

        # Using updated ChatOpenAI and LLMChain for chaining
        llm = ChatOpenAI(model="gpt-4", temperature=0.7)
        chain = LLMChain(llm=llm, prompt=prompt_template)  # Use LLMChain directly
        soundscape_prompt = chain.run(location_description=location_description)  # Use run instead of invoke
        return soundscape_prompt.strip()
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error in GPT-4 prompt generation: {str(e)}")

# 3. Generate audio using Facebook MusicGen
def generate_music(soundscape_prompt):
    try:
        synthesiser = pipeline("text-to-audio", model="facebook/musicgen-medium", device="cpu")

        music = synthesiser(soundscape_prompt, forward_params={"do_sample": True})

        # Convert audio to WAV and return as base64
        output_file = "musicgen_output.wav"
        wavfile.write(output_file, rate=music["sampling_rate"], data=music["audio"])

        # Convert the music file to base64 for sending in response
        with open(output_file, "rb") as f:
            music_base64 = base64.b64encode(f.read()).decode("utf-8")
        
        return music_base64
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error in audio generation: {str(e)}")

# API endpoints
@app.post("/generate-music/")
def generate_music_endpoint(location: Location):
    try:
        # Step 1: Get location description
        location_description = get_location_description(location.latitude, location.longitude)

        # Step 2: Generate soundscape prompt
        soundscape_prompt = generate_soundscape_prompt(location_description)

        # Log the location and prompt
        logging.info(f"Location: {location_description}")
        logging.info(f"Soundscape Prompt: {soundscape_prompt}")

        # Step 3: Generate music
        music_base64 = generate_music(soundscape_prompt)

        return {
            "location_description": location_description,
            "soundscape_prompt": soundscape_prompt,
            "music_file_base64": music_base64,
        }
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Unexpected error: {str(e)}")


# Expose FastAPI app to the web using ngrok
public_url = ngrok.connect(8000)
print(f"FastAPI is running at {public_url}")

# Run the FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)
